## Fuel Tank FEA Analysis
Ian Zabel, ibz@pdx.edu
### Introduction
The purpose of this analysis is to determine the failure criterea of the carbon fiber composite fuel tank walls. An Abaqus 6.14 model is developed assuming that the fuel tank is axisymmetric with some simplifications regarding the internals of the fuel tank. An initial model is developed and run, assuming there is no baffling or plumbing within the fuel tank, to ensure the composite definitions are sensible. A more advanced model is then developed that includes the internals expected to be present.
### Assumptions and Theory
To perform this analysis using a layered composite, the pressure vessel is assumed to be thin-walled. To apply this assumption, the wall thickness $t$ must be much smaller than a characteristic dimension $R$, in this case the vessel radius $r$. The simplifed pressure vessel is assumed to be axisymmetric, and the advanced model vessel is assumed to by symmetric about both the $x,z$ and $y,z$ planes.

In [21]:
import math as m
import numpy as np
import matplotlib
import matplotlib.pyplot as plot
import pylab
%matplotlib inline

# Ian Zabel
# Note: Dimensions from LV4 MDO
#       Atmo pressure from LV4 MDO

# Dimensions

Do   = 0.254   # Outer tank diameter     (m)
Ro   = Do/2    # Outer tank radius       (m)
L    = 1.2     # Overall tank length     (m)
Lc   = L-4*Ro  # Cylindrical tank length (m)
TW   = 10      # Thin-walled assumption
t    = TW*Ro   # Wall thickness          (m)

# Pressure
# Note: Use either np.all() or np.any() for ineq. def

def atmo(x):
    if np.all(x < 11000):
        T = 15.04-0.0065*x
        P = 101.3*((T+273.1)/288.1)**5.26
    elif np.all(11000 <= x) & np.all(x < 25000):
        T = -56.46
        P = 22.65*exp(1.73-0.00016*x) 
    else:
        T = -131.2+0.003*x
        P = 2.488*((T+273.1)/216.6)**(-11.4)

    rho = P/(0.29*(T+273.1)) # Density, ambient air     (kg/m^3)
    Pa  = P*1000             # Pressure, ambient air    (Pa)
    Ta  = T+273.1            # Temperature, ambient air (K)
    return Pa, rho, Ta

x  = np.arange(0, 1e5, 1) # Altitude             (m)
Pa = atmo(x)[0]           # Atmospheric pressure (Pa)
Po = 50-Pa                # Pressure, lox, abs.  (Pa)
Pf = 50-Pa                # Pressure, fuel, abs. (Pa)

# Propellant Parameters
# Note: Volume values TEMP

OF   = 1.3   # Oxidizer to fuel ratio
rhof = 852.3 # Density, ethanol fuel  (kg/m^3)
rhoo = 1141  # Density, lox           (kg/m^3)
Vo   = 1     # Required volume, lox   (m^3)
Vf   = 1     # Required volume, fuel  (m^3)

Endo = 2*(4/3)*m.pi*Ro**3 # End cap volume, lox   (m^3)
Endf = 2*(4/3)*m.pi*Ro**3 # End cap volume, fuel  (m^3)
RVLo = Vo-Endo            # Cylinder volume, lox  (m^3)
RVLf = Vf-Endf            # Cylinder volume, fuel (m^3)
Lcf  = RVLo/(m.pi*Ro**2)  # Cylinder length, lox  (m)
Lco  = RVLf/(m.pi*Ro**2)  # Cylinder length, fuel (m)

# Stress, Cylindrical Portion

Sho = Po*Ro/t # Hoop stress, cylinder, ox    (Pa)
Shf = Pf*Ro/t # Hoop stress, cylinder, fuel  (Pa)
Sao = 0.5*Sho # Axial stress, cylinder, ox   (Pa)
Saf = 0.5*Shf # Axial stress, cylinder, fuel (Pa)

# Stress, Spherical Portion
# Note: Static equilibrium, stress net resultant

Sno = Sao # Stress, endcaps, Ox   (Pa)
Snf = Saf # Stress, endcaps, Fuel (Pa)

print('\n')
print('FUEL TANK PARAMETERS')
print('--------------------')
print('Tank Volume, Ethanol = {0:.1f} m^3'.format(Vf))
print('Tank volume, Lox     = {0:.1f} m^3'.format(Vo))
print('Tank Length, Ethanol = {0:.1f} m'.format(Lcf))
print('Tank Length, Lox     = {0:.1f} m'.format(Lco))

# Pressure Plots

# pylab.rcParams['figure.figsize'] = (10, 10)



FUEL TANK PARAMETERS
--------------------
Tank Volume, Ethanol = 1.0 m^3
Tank volume, Lox     = 1.0 m^3
Tank Length, Ethanol = 19.5 m
Tank Length, Lox     = 19.5 m
